In [18]:
import pandas as pd
import pickle
import os
from sklearn.preprocessing import StandardScaler

# 加载模型
filename = 'loan_approval_model.pkl'
model_dir = 'model'  # 模型存储目录
model_path = os.path.join(model_dir, filename)  # 模型存储路径
loaded_model = pickle.load(open(model_path, 'rb'))

print("模型已加载")

模型已加载


In [ ]:
print('测试模型导入')

hello
